# Reflection Pattern Use

1. **Generation**: AI generates an initial response to a given prompt or task
2. **Reflection**: Model double-checks its own response -- identifies potential errors, inconsistencies or areas for improvement
3. **Iterative Refinement**: Based on self-assessment, model correctly fixes any mistakes to enhance quality and accuracy

In [ ]:
# Step 1: Setup
!pip install --upgrade openai
import openai
from openai import OpenAI

client = OpenAI(api_key="INSERT API KEY HERE")

# Step 2: Input your claim
claim = "COVID-19 vaccines alter your DNA."  # You can replace this

# Step 3: PLAN
def generate_plan(claim):
    plan_prompt = f"""
You are an intelligent planning agent. Your job is to verify claims using a strategic, step-by-step method.

Here is the claim:
"{claim}"

Devise a verification plan that outlines:
1. What types of sources or knowledge are needed
2. What reasoning steps should be taken
3. What signs of misinformation to look for

Respond only with the plan.
"""
    response = client.chat.completions.create(
        model="gpt-4",
        messages=[{"role": "user", "content": plan_prompt}]
    )
    return response.choices[0].message.content

# Step 4: ACT
def execute_fact_check(claim, plan):
    act_prompt = f"""
Using this plan:
{plan}

Fact-check the following claim:
"{claim}"

Provide:
1. A truthful answer
2. Justification using scientific or logical reasoning
3. A confidence level (High, Medium, Low)

Do not fabricate data or cite made-up studies. Stick to what is publicly known.
"""
    response = client.chat.completions.create(
        model="gpt-4",
        messages=[{"role": "user", "content": act_prompt}]
    )
    return response.choices[0].message.content

# Step 5: REFLECT
def reflect(plan, result):
    reflect_prompt = f"""
You used this plan:
{plan}

And arrived at this result:
{result}

Now reflect on:
1. What was done well?
2. What could be improved in the reasoning or sources?
3. Should your confidence level change?

Output a reflection + improved plan if needed.
"""
    response = client.chat.completions.create(
        model="gpt-4",
        messages=[{"role": "user", "content": reflect_prompt}]
    )
    return response.choices[0].message.content

# Step 6: Run the reflection pattern loop
plan = generate_plan(claim)
print("🧠 PLAN:\n", plan)

result = execute_fact_check(claim, plan)
print("\n📘 FACT-CHECK RESULT:\n", result)

reflection = reflect(plan, result)
print("\n🔍 REFLECTION:\n", reflection)



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 764.4/764.4 kB 8.8 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.97.0
    Uninstalling openai-1.97.0:
      Successfully uninstalled openai-1.97.0
🧠 PLAN:
 Verification Plan:

1. Types of Sources or Knowledge Needed:
    - Scientific journals and articles on COVID-19 vaccine research.
    - Statements, briefings or information released by reliable health organizations such as the World Health Organization (WHO), Centers for Disease Control and Prevention (CDC), National Institutes of Health (NIH), and European Medicine Agency (EMA).
    - Virology and genetics textbooks or educational resources.
    - Clinical trial data on COVID-19 vaccines.
    - Expert opinions and interviews from virologists and geneticists.

2. Reasoning Steps to be taken:
    - Conduct an in-depth review of reputable scientific literature concerning the functioning of COVID-19 vaccines, focusing particularly on how mRNA and ve

# Tool Use Pattern

 1. **Select Tool** – Pick based on task requirements
 2. **Access Tool** – Install, import, or open
 3. **Configure** – Set up parameters or environment
 4. **Execute** – Run the tool on a task or dataset
 5. **Monitor** – Observe outputs, logs, errors
 6. **Evaluate** – Check if it solved the problem effectively
 7. **Iterate** – Refine settings or switch tools if needed

In [ ]:
import json

# ── mock “external” tools ────────────────────────────────────────────────────
def search_population_tool(country, year):
    population_db = {
        ("China", 2014): "1.364 billion",
        ("China", 2024): "1.412 billion",
        ("India", 2014): "1.296 billion",
        ("India", 2024): "1.429 billion",
    }
    return population_db.get((country, year), "Data not available")

def calculator_tool(value_current, value_past):
    """
    Returns (absolute_difference_in_billions, percent_change).
    Expects strings like '1.364 billion'.
    """
    try:
        now = float(value_current.split()[0])
        then = float(value_past.split()[0])
        diff = now - then
        pct  = (diff / then) * 100
        return round(diff, 3), round(pct, 2)
    except Exception as exc:
        raise ValueError(f"Bad input to calculator_tool: {exc}")

# ── agent driver ─────────────────────────────────────────────────────────────
def agent_compare_population(country1, country2, year_start, year_end):
    """
    Compare absolute population growth between two countries over a period.
    """
    tools = [
        {
            "type": "function",
            "name": "search_population_tool",
            "description": "Get the population for a given country in a given year",
            "parameters": {
                "type": "object",
                "properties": {
                    "country": {"type": "string"},
                    "year": {"type": "number"}
                },
                "required": ["country", "year"],
                "additionalProperties": False
            },
            "strict": True
        },
        {
            "type": "function",
            "name": "calculator_tool",
            "description": "Compute absolute and percent change between two population strings",
            "parameters": {
                "type": "object",
                "properties": {
                    "value_current": {"type": "string"},
                    "value_past":    {"type": "string"}
                },
                "required": ["value_current", "value_past"],
                "additionalProperties": False
            },
            "strict": True
        }
    ]

    # STEP 1 – fetch populations
    pop1_start = search_population_tool(country1, year_start)
    pop1_end   = search_population_tool(country1, year_end)
    pop2_start = search_population_tool(country2, year_start)
    pop2_end   = search_population_tool(country2, year_end)

    # STEP 2 – calculate growth via calculator_tool
    growth1_abs, growth1_pct = calculator_tool(pop1_end, pop1_start)
    growth2_abs, growth2_pct = calculator_tool(pop2_end, pop2_start)

    # STEP 3 – decide the winner
    if growth1_abs > growth2_abs:
        winner = country1
    elif growth2_abs > growth1_abs:
        winner = country2
    else:
        winner = "Both grew equally"

    # STEP 4 – report
    print(f"{country1}: +{growth1_abs} billion ({growth1_pct} %)")
    print(f"{country2}: +{growth2_abs} billion ({growth2_pct} %)")
    print(f"Higher absolute growth: {winner}")

# ── run a quick check ────────────────────────────────────────────────────────
agent_compare_population("China", "India", 2014, 2024)



China: +0.048 billion (3.52 %)
India: +0.133 billion (10.26 %)
Higher absolute growth: India


# Planning Pattern Agent

 1. Define goal
 2. Break into tasks
 3. Set priorities
 4. Allocate time/resources
 5. Set checkpoints or deadlines
 6. Review & adjust plan as needed

In [ ]:
!pip install --upgrade openai

import openai
from openai import OpenAI

client = OpenAI(api_key="INSERT API KEY HERE")

def generate_research_plan(topic):
    prompt = f"""
You are a smart research planning agent. Your goal is to research:
"{topic}"

1. Break this task into 3–5 logical steps.
2. Each step should include:
   - What to research
   - Why it's needed
   - The intended output

Output only the plan.
"""
    response = client.chat.completions.create(
        model="gpt-4",
        messages=[{"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content
def execute_step(step_instruction):
    prompt = f"""
You are a research assistant. Based on this instruction:
"{step_instruction}"

Perform the task using your internal knowledge.
Give a clear and concise result.
"""
    response = client.chat.completions.create(
        model="gpt-4",
        messages=[{"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content
# Define your research topic
topic = "Top 3 challenges in using AI for healthcare diagnostics"

# Generate a structured plan
print(" Generating plan...")
plan_text = generate_research_plan(topic)
print(plan_text)

# Split plan into steps
print("\n Executing steps...\n")
steps = [line.strip() for line in plan_text.split('\n') if line.startswith("Step")]
step_outputs = []

for step in steps:
    print(f"▶ {step}")
    output = execute_step(step)
    step_outputs.append((step, output))
    print(f" Output:\n{output}\n")

print("Final Summary Report\n")
for i, (step, output) in enumerate(step_outputs):
    print(f"{step}\n{output}\n{'-'*60}\n")



 Generating plan...
Step 1: Understand the Concept
- What to Research: Start with a study on the concept of AI in healthcare diagnostics, its applications, recent advancements and the transformation it has brought in the healthcare industry.
- Why: This is essential to gain an overview and familiar understanding of the subject matter which will aid in pinpointing potential challenges effectively.
- Intended Output: A comprehensive understanding of AI in healthcare diagnostics including its applications and benefits.

Step 2: Identify Potential Challenges
- What to Research: After understanding the basics, the next step is to identify potential challenges that are associated with using AI in healthcare diagnostics. This can be done by reading scientific articles, technology reports, and publications from credible sources.
- Why: This is crucial in addressing the key research objective and analyzing the complexities in AI integration.
- Intended Output: A list of potential challenges tha